In [1]:
import pandas as pd

In [2]:
# Read the population data file and skip empty rows
pop_filename = '../Data/NUTS-3_Population.xlsx'
no_data_rows = [0, 1, 2, 3, 4, 5, 6, 7, 8, 10, 2093, 2094, 2095,
                 2096, 2097, 2098, 2099, 2100, 2101, 2102]
pop = pd.read_excel(pop_filename, skiprows=no_data_rows)

# Preprocess the population file
# Skip unnamed columns with no population data
named_columns = [col for col in pop.columns if not col.startswith('Unnamed')]
pop = pd.read_excel(pop_filename, skiprows=no_data_rows, usecols=named_columns)
pop.rename(columns={'TIME': 'NUTS Code', 'TIME.1': 'NUTS Desc'}, inplace=True)
pop

,NUTS Code,NUTS Desc,1990,1991,1992,1993,1994,1995,1996,1997,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,EU27_2020,European Union - 27 countries (from 2020),418030739,419504036,420413536,421912578,423104986,423960534,424641798,425273491,...,442883888,443666812,444802830,445534430,446208557,446446444,447319916,447073916,446735291,448753823
1,EU28,European Union - 28 countries (2013-2020),475187711,476842235,477925130,479561788,480893003,481904006,482736385,483512803,...,507235091,508520205,510181874,511378572,512372000,513093556,:,:,:,:
2,EU27_2007,European Union - 27 countries (2007-2013),470415155,472060056,473329265,475006017,476247848,477245113,478155218,478979775,...,502988282,504294889,505991205,507224359,508266507,509017310,:,:,:,:
3,BE,Belgium,9947782,9986975,10021997,10068319,10100631,10130574,10143047,10170226,...,11180840,11237274,11311117,11351727,11398589,11455519,11522440,11554767,11617623,11742796
4,BE1,Région de Bruxelles-Capitale/Brussels Hoofdste...,:,:,:,:,:,:,:,:,...,1172751,1184101,1201285,1199095,1205492,1215289,1223364,1226329,1228655,1253178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2077,TRC3,"Mardin, Batman, Şırnak, Siirt",:,:,:,:,:,:,:,:,...,2116727,2153921,2173759,2179588,2222601,2284158,2307332,2343826,2367645,2393781
2078,TRC31,Mardin,:,:,:,:,:,:,:,:,...,779738,788996,796591,796237,809719,829195,838778,854716,862757,870374
2079,TRC32,Batman,:,:,:,:,:,:,:,:,...,547581,557593,566633,576899,585252,599103,608659,620278,626319,634491
2080,TRC33,Şırnak,:,:,:,:,:,:,:,:,...,475255,488966,490184,483788,503236,524190,529615,537762,546589,557605


In [3]:
# Load cities file
c_filename = '../Data/AirportCodes_Cities_NUTS.xlsx'
cities = pd.read_excel(c_filename)
cities

,Airport Code,NUTS_3_code,City,Country
0,EBAW,BE211,Antwerp,Belgium
1,EBBR,BE100,Brussels,Belgium
2,EBCI,BE322,Charleroi,Belgium
3,EBLG,BE332,Liege,Belgium
4,EBOS,BE255,Ostend,Belgium
...,...,...,...,...
99,LFRS,FRG01,Nantes,France
100,LFSB,FRF12,Basel/Mulhouse,France/Switzerland
101,LFST,FRF11,Strasbourg,France
102,LFTH,FRL05,Toulon,France


In [4]:
cities = cities.merge(pop, left_on='NUTS_3_code', right_on='NUTS Code', how='left')
cities = cities.drop(columns=['NUTS Code', 'NUTS Desc'])

output_filepath = '../Data/AirportCodes_Cities_NUTS_Population.xlsx'
cities.to_excel(output_filepath)